In [1]:
import numpy as np
import pandas as pd
import torch
from transformers import BertTokenizer, BertForQuestionAnswering

## Load data from the previous step

In [2]:
question_list = pd.read_csv('task4_question1_question_keyword.csv', usecols=[2, 3]).drop_duplicates()
questions = question_list.set_index('category').to_dict()['question']

In [3]:
sentence_candidates = pd.read_csv('ALLRanking_TASK4_Q1_TOP_1000.csv')
abstracts = pd.read_csv('ALLRanking_TASK4_Q1_TOP_1000_abstract.csv', usecols=[1,3]).set_index('pid').to_dict()['abstract']

In [4]:
sentence_candidates['abstract'] = sentence_candidates['pid'].map(abstracts)

In [5]:
sentence_candidates['question'] = sentence_candidates['category'].map(questions)

In [6]:
sentence_candidates

,Unnamed: 0,pid,newpid,category,sent,newscore,abstract,question
0,180179,cszqykpu,246,antihiv,"Interestingly, lopinavir, ritonavir, and darun...",0.808423,The infection of a novel coronavirus found in ...,Can anti HIV drug cure COVID-2019
1,180181,qdt90c22,31,antihiv,Our results showed that several HIV inhibitors...,0.806379,Direct-acting antivirals are effective tools t...,Can anti HIV drug cure COVID-2019
2,180173,vbgf50os,192,antihiv,"Ritonavir, lopinavir and darunavir were then d...",0.805869,"Three anti-HIV drugs, ritonavir, lopinavir and...",Can anti HIV drug cure COVID-2019
3,180189,vbgf50os,190,antihiv,"Three anti-HIV drugs, ritonavir, lopinavir and...",0.805594,"Three anti-HIV drugs, ritonavir, lopinavir and...",Can anti HIV drug cure COVID-2019
4,180176,qmg58cgr,128,antihiv,The focused drug repurposing of known approved...,0.803729,The focused drug repurposing of known approved...,Can anti HIV drug cure COVID-2019
...,...,...,...,...,...,...,...,...
7995,7265,7wfdwou8,11864,receptor,It has been known that SARS coronavirus (SARS_...,0.164469,Severe acute respiratory syndrome (SARS) is a...,What Kinds of receptor are involved in COVID-...
7996,1021,ebwxryai,12866,receptor,Different forms of lung injury can activate se...,0.164468,Different forms of lung injury can activate se...,What Kinds of receptor are involved in COVID-...
7997,6439,ck0jpp7e,24472,receptor,"Within this mutant gene, we defined a G-rich s...",0.164465,We investigated a herpesvirus mutant that con...,What Kinds of receptor are involved in COVID-...
7998,6412,nn9gj0z1,3892,receptor,A variety of perturbants were employed to char...,0.164459,Respiratory Syncytial Virus (RSV) is a highly ...,What Kinds of receptor are involved in COVID-...


## Load pre-trained model and define QA function

In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
qa_tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
qa_model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad').to(device)

qa_model.eval()
assert not qa_model.training

In [24]:
def bert_squad_qa(question, document, model, tokenizer, device):
    encoded = tokenizer.encode_plus(question, document, return_tensors='pt', max_length=512)
    start_scores, end_scores = model(encoded['input_ids'].to(device),
                                     token_type_ids=encoded['token_type_ids'].to(device))

    tokens = tokenizer.convert_ids_to_tokens(encoded['input_ids'].squeeze())
    ans_start, ans_end = torch.argmax(start_scores), torch.argmax(end_scores)
    
    ans_tokens = tokens[ans_start: ans_end+1]
    if '[SEP]' in ans_tokens:
        ans_tokens = ans_tokens[ans_tokens.index('[SEP]')+1:]
    ans = tokenizer.convert_tokens_to_string(ans_tokens)
    ans = ans.replace(' - ', '-').replace('[CLS]', '')

    return ans

In [25]:
q = 'What Kinds of  receptor are involved in COVID-2019? '
d = 'Angiotensin-converting enzyme 2 (ACE2) degrades angiotensin (Ang) II to Ang-(1–7), and protects against diabetic renal injury. Soluble ACE2 fragments are shed from the proximal tubule, and appear at high levels in the urine with diabetes. High glucose-induced shedding of ACE2 from proximal tubular cells is mediated by the enzyme “a disintegrin and metalloproteinase-17″ (ADAM17). Here, we investigated the mechanism for constitutive shedding of ACE2. Mouse proximal tubular cells were cultured and ACE2 shedding into the media was assessed by enzyme activity assay and immunoblot analysis. Cells were incubated with pharmacologic inhibitors, or transfected with silencing (si) RNA. Incubation of proximal tubular cells with increasing concentrations of D-glucose stimulated ACE2 shedding, which peaked at 16 mM, while L-glucose (osmotic control) had no effect on shedding. In cells maintained in 7.8 mM D-glucose, ACE2 shedding was significantly inhibited by the pan-protein kinase C (PKC) competitive inhibitor sotrastaurin, but not by an inhibitor of ADAM17. Incubation of cells with the PKC-α and -β1-specific inhibitor Go6976, the PKC β1 and β2-specific inhibitor ruboxistaurin, inhibitors of matrix metalloproteinases-2,-8, and -9, or an inhibitor of ADAM10 (GI250423X) had no effect on basal ACE2 shedding. By contrast, the PKC-δ inhibitor rottlerin significantly inhibited both constitutive and high glucose-induced ACE2 shedding. Transfection of cells with siRNA directed against PKC-δ reduced ACE2 shedding by 20%, while knockdown of PKC-ε was without effect. These results indicate that constitutive shedding of ACE2 from proximal tubular cells is mediated by PKC-δ, which is also linked to high glucose-induced shedding. Targeting PKC-δ may preserve membrane-bound ACE2 in proximal tubule in disease states and diminish Ang II-stimulated adverse signaling.'

bert_squad_qa(q, d, qa_model ,qa_tokenizer, device)

''

## generate answers from given sentences

In [ ]:
sentence_candidates['answer'] = sentence_candidates.apply(lambda x: bert_squad_qa(x['question'], x['sent'], qa_model, qa_tokenizer, device)[0], axis=1)

In [ ]:
for _, row in sentence_candidates_examples.iterrows():
    print('Question:', row['question'])
    print('Sentence:', row['sent'])
    print('Answer: ', row['answer'])
    print('\n\n')

## Generate answers from abstract

In [11]:
# First merge the papers for each question by summing up the scores for sentences
# The more relevant sentences detected, the more informative the abstract may be.
abstract_candidates = sentence_candidates.groupby(['question', 'abstract']).agg({'newscore': sum})

In [12]:
abstract_candidates = abstract_candidates.reset_index().sort_values(by=['question', 'newscore'], ascending=False)

In [13]:
question_list = abstract_candidates['question'].unique().tolist()

In [14]:
q = question_list[0]
df = abstract_candidates[abstract_candidates['question']==q]

In [17]:
print(q, '\n')
row_count = 0
for x, row in df.iterrows():
    ans = bert_squad_qa(q.replace('  ', ' '), row['abstract'], qa_model, qa_tokenizer, device)
    print(row['abstract'])
    print()
    print(ans)
    print('\n\n\n')

What Kinds of  receptor are involved in COVID-2019? 

Angiotensin-converting enzyme 2 (ACE2) is highly expressed in the kidney proximal tubule, where it cleaves angiotensin (Ang) II to Ang-(1-7). Urinary ACE2 levels increase in diabetes, suggesting that ACE2 may be shed from tubular cells. The aim of this study was to determine if ACE2 is shed from proximal tubular cells, to characterize ACE2 fragments, and to study pathways for shedding. Studies involved primary cultures of mouse proximal tubular cells, with ACE2 activity measured using a synthetic substrate, and analysis of ACE2 fragments by immunoblots and mass spectrometry. The culture media from mouse proximal tubular cells demonstrated a time-dependent increase in ACE2 activity, suggesting constitutive ACE2 shedding. ACE2 was detected in media as two bands at ∼90 kDa and ∼70 kDa on immunoblots. By contrast, full-length ACE2 appeared at ∼100 kDa in cell lysates or mouse kidney cortex. Mass spectrometry of the two deglycosylated fr

Angiotensin-converting enzyme 2 (ACE2) is expressed in the kidney and may be a renoprotective enzyme, since it converts angiotensin (Ang) II to Ang-(1-7). ACE2 has been detected in urine from patients with chronic kidney disease. We measured urinary ACE2 activity and protein levels in renal transplant patients (age 54 yrs, 65% male, 38% diabetes, n = 100) and healthy controls (age 45 yrs, 26% male, n = 50), and determined factors associated with elevated urinary ACE2 in the patients. Urine from transplant subjects was also assayed for ACE mRNA and protein. No subjects were taking inhibitors of the renin-angiotensin system. Urinary ACE2 levels were significantly higher in transplant patients compared to controls (p = 0.003 for ACE2 activity, and p≤0.001 for ACE2 protein by ELISA or western analysis). Transplant patients with diabetes mellitus had significantly increased urinary ACE2 activity and protein levels compared to non-diabetics (p<0.001), while ACE2 mRNA levels did not differ. U

Heart failure remains the most common cause of death and disability, and a major economic burden, in industrialized nations. Physiological, pharmacological, and clinical studies have demonstrated that activation of the renin-angiotensin system is a key mediator of heart failure progression. Angiotensin converting enzyme 2 (ACE2), a homologue of ACE, is a monocarboxypeptidase that converts angiotensin II (Ang II) into angiotensin 1–7 (Ang 1–7) which, by virtue of its actions on the Mas receptor, opposes the molecular and cellular effects of Ang II. ACE2 is widely expressed in cardiomyocytes, cardiofibroblasts, and coronary endothelial cells. Recent preclinical translational studies confirmed a critical counter-regulatory role of ACE2/Ang 1–7 axis on the activated renin-angiotensin system that results in heart failure with preserved ejection fraction. While loss of ACE2 enhances susceptibility to heart failure, increasing ACE2 level prevents and reverses the heart failure phenotype. ACE2

Serum or plasma proteases have been associated with various diseases including cancer, inflammation, or reno-cardiovascular diseases. We aimed to investigate whether the enzymatic activities of serum proteases are associated with the estimated glomerular filtration rate (eGFR) in patients with different stages of chronic kidney disease (CKD). Our study population comprised 268 participants of the “Greifswald Approach to Individualized Medicine” (GANI_MED) cohort. Enzymatic activity of aminopeptidase A, aminopeptidase B, alanyl (membrane) aminopeptidase, insulin-regulated aminopeptidase, puromycin-sensitive aminopeptidase, leucine aminopeptidase 3, prolyl-endopeptidase (PEP), dipeptidyl peptidase 4 (DPP4), angiotensin I-converting enzyme, and angiotensin I-converting enzyme 2 (ACE2) proteases was measured in serum. Linear regression of the respective protease was performed on kidney function adjusted for age and sex. Kidney function was modeled either by the continuous Modification of D

 A purified recombinant spike (S) protein was studied for its effect on stimulating human peripheral blood monocyte macrophages (PBMC). We examined inflammatory gene mRNA abundances found in S protein-treated PBMC using gene arrays. We identified differential mRNA abundances of genes with functional properties associated with antiviral (CXCL10) and inflammatory (IL-6 and IL-8) responses. We confirmed cytokine mRNA increases by real-time quantitative(q) RT-PCR or ELISA. We further analyzed the sensitivity and specificity of the prominent IL-8 response. By real-time qRT-PCR, S protein was shown to stimulate IL-8 mRNA accumulation in a dose dependent manner while treatment with E protein did not. Also, titration of S protein-specific production and secretion of IL-8 by ELISA showed that the dose of 5.6nM of S produced a significant increase in IL-8 (p =0.003) compared to mock-treated controls. The increase in IL-8 stimulated by a concentration of 5.6nM of S was comparable to concentration

...

The spontaneous and reversible formation of foci and filaments that contain proteins involved in different metabolic processes is common in both the nucleus and the cytoplasm. Stress granules (SGs) and processing bodies (PBs) belong to a novel family of cellular structures collectively known as mRNA silencing foci that harbour repressed mRNAs and their associated proteins. SGs and PBs are highly dynamic and they form upon stress and dissolve thus releasing the repressed mRNAs according to changes in cell physiology. In addition, aggregates containing abnormal proteins are frequent in neurodegenerative disorders. In spite of the growing relevance of these supramolecular aggregates to diverse cellular functions a reliable automated tool for their systematic analysis is lacking. Here we report a MATLAB Script termed BUHO for the high-throughput image analysis of cellular foci. We used BUHO to assess the number, size and distribution of distinct objects with minimal deviation from manually

The interferon (IFN)-inducible 2′-5′-oligoadenylate synthetase (OAS)/RNase L pathway blocks infections by some types of viruses through cleavage of viral and cellular single-stranded RNA. Viruses induce type I IFNs that initiate signaling to the OAS genes. OAS proteins are pathogen recognition receptors for the viral pathogen-associated molecular pattern, double-stranded RNA. Double-stranded RNA activates OAS to produce p(x)5′A(2′p5′A)(n); x = 1–3; n > 2 (2-5A) from ATP. Upon binding 2-5A, RNase L is converted from an inactive monomer to a potently active dimeric endoribonuclease for single-stranded RNA. RNase L contains, from N- to C-terminus, a series of 9 ankyrin repeats, a linker, several protein kinase-like motifs, and a ribonuclease domain homologous to Ire1 (involved in the unfolded protein response). In the past few years, it has become increasingly apparent that RNase L and OAS contribute to innate immunity in many ways. For example, small RNA cleavage products produced by RNa

 The giant panda (Ailuropoda melanoleuca) is an endangered species. Interleukin-18 (IL-18) plays an important role in the innate and adaptive immune responses by inducing IFN-γ. IL-18 has been implicated in the pathogenesis of various diseases. IL-18 binding protein (IL-18BP) is an intrinsic inhibitor of IL-18 that possesses higher affinity to IL-18. In this study, we cloned and characterized IL-18BP in giant panda (AmIL-18BP) from the spleen. The amino acid sequence of giant panda IL-18BP ORF shared about 65% identities with other species. To evaluate the effects of AmIL-18BP on the immune responses, we expressed the recombinant AmIL-18BP in Escherichia coli BL21 (DE3).The fusing protein PET-AmIL-18BP was purified by nickel affinity column chromatography. The biological function of purified PET-AmIL-18BP was determined on mice splenocyte by qRT-PCR. The results showed that AmIL-18BP was functional and could significantly reduce IFN-γ production in murine splenocytes. These results wil

Viral infection triggers a series of signaling cascades, which converge to activate the transcription factors nuclear factor-κB (NF-κB) and interferon regulatory factor 3 (IRF3), thereby inducing the transcription of type I interferons (IFNs). Although not fully characterized, these innate antiviral responses are fine-tuned by dynamic ubiquitination and deubiquitination processes. In this study, we report ubiquitin-specific protease (USP) 15 is involved in regulation of the retinoic acid-inducible gene I (RIG-I)-dependent type I IFN induction pathway. Knockdown of endogenous USP15 augmented cellular antiviral responses. Overexpression of USP15 inhibited the transcription of IFN-β. Further analyses identified histidine 862 as a critical residue for USP15’s catalytic activity. Interestingly, USP15 specifically removed lysine 63-linked polyubiquitin chains from RIG-I among the essential components in RIG-I-like receptor-dependent pathway. In addition, we demonstrated that in contrast to U

KeyboardInterrupt: 